# Real Estate Price Prediction Engine - Technical Challenge

## Objective
Build a production-ready ML model to predict property transaction prices (TRANS_VALUE) with high accuracy and explainability.

## Approach Overview
1. **EDA**: Understand data quality, distributions, and business patterns
2. **Feature Engineering**: Create predictive features while avoiding data leakage
3. **Model Development**: Train CatBoost with log-transformed target and MAE loss
4. **Evaluation**: Assess performance across price ranges and property segments
5. **Interpretability**: Use SHAP to explain predictions and generate business insights

## Key Technical Decisions
- **Model**: CatBoost (handles categorical features natively, robust to outliers)
- **Target Transform**: log1p (handles skewed price distribution)
- **Loss Function**: MAE (robust to outliers)
- **Split Strategy**: Chronological 70/15/15 (prevents temporal leakage)
- **Feature Strategy**: Train-only aggregates with unseen category handling

---
## 1. Setup and Data Loading

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set visualization style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

# For reproducibility
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

print("Libraries imported successfully")

In [ ]:
# Load the dataset
df = pd.read_csv('../transactions-2025-03-21.csv')

print(f"Dataset loaded: {df.shape[0]:,} rows × {df.shape[1]} columns")
print(f"\nMemory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

---
## 2. Exploratory Data Analysis (EDA)

### 2.1 Dataset Overview

In [ ]:
# Basic information
print("=" * 80)
print("DATASET STRUCTURE")
print("=" * 80)
df.info()

In [ ]:
# First few rows
print("\nFirst 5 rows:")
df.head()

In [ ]:
# Column data types and descriptions
print("\n" + "=" * 80)
print("COLUMN ANALYSIS")
print("=" * 80)

column_info = pd.DataFrame({
    'Column': df.columns,
    'Type': df.dtypes,
    'Non-Null Count': df.count(),
    'Null Count': df.isnull().sum(),
    'Null %': (df.isnull().sum() / len(df) * 100).round(2),
    'Unique Values': df.nunique()
})

column_info

In [ ]:
# Parse date column to understand temporal range
df['INSTANCE_DATE'] = pd.to_datetime(df['INSTANCE_DATE'])

print("\n" + "=" * 80)
print("TEMPORAL COVERAGE")
print("=" * 80)
print(f"Date Range: {df['INSTANCE_DATE'].min()} to {df['INSTANCE_DATE'].max()}")
print(f"Time Span: {(df['INSTANCE_DATE'].max() - df['INSTANCE_DATE'].min()).days} days")
print(f"\nTransactions by Year:")
print(df['INSTANCE_DATE'].dt.year.value_counts().sort_index())

### 2.2 Missing Values and Data Quality

In [ ]:
# Detailed missing value analysis
missing_analysis = pd.DataFrame({
    'Column': df.columns,
    'Missing_Count': df.isnull().sum(),
    'Missing_Percentage': (df.isnull().sum() / len(df) * 100).round(2),
    'Empty_Strings': (df == '').sum(),
    'Empty_String_Percentage': ((df == '').sum() / len(df) * 100).round(2)
})

missing_analysis = missing_analysis[
    (missing_analysis['Missing_Count'] > 0) | (missing_analysis['Empty_Strings'] > 0)
].sort_values('Missing_Percentage', ascending=False)

print("\n" + "=" * 80)
print("MISSING VALUES & DATA QUALITY ISSUES")
print("=" * 80)
if len(missing_analysis) > 0:
    display(missing_analysis)
else:
    print("No missing values found!")

# Check for empty strings in categorical columns
categorical_cols = df.select_dtypes(include=['object']).columns
print(f"\n\nEmpty string counts in categorical columns:")
for col in categorical_cols:
    empty_count = (df[col] == '').sum()
    if empty_count > 0:
        print(f"{col}: {empty_count} ({empty_count/len(df)*100:.2f}%)")

### 2.3 Target Variable Analysis (TRANS_VALUE)

Understanding the distribution of our target variable is crucial for:
- Choosing appropriate transformations
- Identifying outliers
- Setting realistic performance expectations

In [ ]:
# Statistical summary of TRANS_VALUE
print("=" * 80)
print("TARGET VARIABLE (TRANS_VALUE) STATISTICS")
print("=" * 80)
print(df['TRANS_VALUE'].describe())
print(f"\nSkewness: {df['TRANS_VALUE'].skew():.2f}")
print(f"Kurtosis: {df['TRANS_VALUE'].kurtosis():.2f}")

# Check for invalid values
invalid_prices = (df['TRANS_VALUE'] <= 0).sum()
print(f"\nInvalid prices (<=0): {invalid_prices} ({invalid_prices/len(df)*100:.2f}%)")

In [ ]:
# Visualize TRANS_VALUE distribution - Raw vs Log-transformed
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Raw distribution
axes[0, 0].hist(df['TRANS_VALUE'], bins=100, edgecolor='black', alpha=0.7)
axes[0, 0].set_title('TRANS_VALUE Distribution (Raw)', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Transaction Value')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].axvline(df['TRANS_VALUE'].median(), color='red', linestyle='--', label=f'Median: {df["TRANS_VALUE"].median():,.0f}')
axes[0, 0].legend()

# Log-transformed distribution
axes[0, 1].hist(np.log1p(df['TRANS_VALUE']), bins=100, edgecolor='black', alpha=0.7, color='green')
axes[0, 1].set_title('TRANS_VALUE Distribution (Log1p)', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Log(Transaction Value + 1)')
axes[0, 1].set_ylabel('Frequency')

# Boxplot - Raw
axes[1, 0].boxplot(df['TRANS_VALUE'], vert=False)
axes[1, 0].set_title('TRANS_VALUE Boxplot (Raw)', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('Transaction Value')

# Boxplot - Log
axes[1, 1].boxplot(np.log1p(df['TRANS_VALUE']), vert=False)
axes[1, 1].set_title('TRANS_VALUE Boxplot (Log1p)', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('Log(Transaction Value + 1)')

plt.tight_layout()
plt.show()

print("\n📊 OBSERVATION: Compare the distributions above.")
print("   - Raw distribution is highly right-skewed (long tail of expensive properties)")
print("   - Log transformation normalizes the distribution, making it more suitable for modeling")
print("   - This justifies using log1p(TRANS_VALUE) as our target during training")

### 2.4 Numerical Features Analysis

In [ ]:
# Analyze numerical columns
numerical_cols = ['TRANS_VALUE', 'PROCEDURE_AREA', 'ACTUAL_AREA', 'ROOMS_EN', 
                  'PARKING', 'TOTAL_BUYER', 'TOTAL_SELLER']

print("=" * 80)
print("NUMERICAL FEATURES SUMMARY")
print("=" * 80)
df[numerical_cols].describe().T

In [ ]:
# Check for data quality issues in numerical columns
print("\n" + "=" * 80)
print("NUMERICAL DATA QUALITY CHECKS")
print("=" * 80)

for col in ['ACTUAL_AREA', 'PROCEDURE_AREA']:
    zero_or_negative = (df[col] <= 0).sum()
    print(f"\n{col}:")
    print(f"  - Zero or negative values: {zero_or_negative} ({zero_or_negative/len(df)*100:.2f}%)")
    print(f"  - Min: {df[col].min()}, Max: {df[col].max()}")
    print(f"  - Median: {df[col].median():.2f}")

In [ ]:
# Compute price per sqft for analysis (NOT as a model feature to avoid leakage)
# This helps us understand the data and identify potential outliers
df['price_per_sqft_temp'] = df['TRANS_VALUE'] / df['ACTUAL_AREA'].replace(0, np.nan)

print("\n" + "=" * 80)
print("PRICE PER SQFT ANALYSIS (for EDA only, NOT a model feature)")
print("=" * 80)
print(df['price_per_sqft_temp'].describe())

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

axes[0].hist(df['price_per_sqft_temp'].dropna(), bins=100, edgecolor='black', alpha=0.7)
axes[0].set_title('Price per Sqft Distribution', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Price per Sqft')
axes[0].set_ylabel('Frequency')

axes[1].boxplot(df['price_per_sqft_temp'].dropna(), vert=False)
axes[1].set_title('Price per Sqft Boxplot', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Price per Sqft')

plt.tight_layout()
plt.show()

### 2.5 Categorical Features Analysis

In [ ]:
# Analyze cardinality of categorical features
categorical_cols = ['GROUP_EN', 'PROCEDURE_EN', 'IS_OFFPLAN_EN', 'IS_FREE_HOLD_EN',
                   'USAGE_EN', 'AREA_EN', 'PROP_TYPE_EN', 'PROP_SB_TYPE_EN',
                   'NEAREST_METRO_EN', 'NEAREST_MALL_EN', 'NEAREST_LANDMARK_EN',
                   'MASTER_PROJECT_EN', 'PROJECT_EN']

print("=" * 80)
print("CATEGORICAL FEATURES CARDINALITY")
print("=" * 80)

cardinality_df = pd.DataFrame({
    'Column': categorical_cols,
    'Unique_Values': [df[col].nunique() for col in categorical_cols],
    'Cardinality_Ratio': [df[col].nunique() / len(df) for col in categorical_cols]
}).sort_values('Unique_Values', ascending=False)

cardinality_df['Cardinality_Type'] = cardinality_df['Unique_Values'].apply(
    lambda x: 'Low (<10)' if x < 10 else ('Medium (10-100)' if x < 100 else 'High (100+)')
)

display(cardinality_df)

print("\n📊 OBSERVATION:")
print("   - High cardinality features (PROJECT_EN, AREA_EN, etc.) need special handling")
print("   - CatBoost can handle these natively without one-hot encoding")
print("   - We'll create aggregated features from these for additional predictive power")

In [ ]:
# Visualize top categories for selected features
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# PROP_TYPE_EN
top_prop_types = df['PROP_TYPE_EN'].value_counts().head(10)
axes[0, 0].barh(range(len(top_prop_types)), top_prop_types.values)
axes[0, 0].set_yticks(range(len(top_prop_types)))
axes[0, 0].set_yticklabels(top_prop_types.index)
axes[0, 0].set_title('Top 10 Property Types', fontsize=12, fontweight='bold')
axes[0, 0].set_xlabel('Count')

# AREA_EN
top_areas = df['AREA_EN'].value_counts().head(10)
axes[0, 1].barh(range(len(top_areas)), top_areas.values)
axes[0, 1].set_yticks(range(len(top_areas)))
axes[0, 1].set_yticklabels(top_areas.index)
axes[0, 1].set_title('Top 10 Areas', fontsize=12, fontweight='bold')
axes[0, 1].set_xlabel('Count')

# IS_OFFPLAN_EN
offplan_counts = df['IS_OFFPLAN_EN'].value_counts()
axes[1, 0].bar(range(len(offplan_counts)), offplan_counts.values)
axes[1, 0].set_xticks(range(len(offplan_counts)))
axes[1, 0].set_xticklabels(offplan_counts.index)
axes[1, 0].set_title('Off-Plan vs Ready Properties', fontsize=12, fontweight='bold')
axes[1, 0].set_ylabel('Count')

# IS_FREE_HOLD_EN
freehold_counts = df['IS_FREE_HOLD_EN'].value_counts()
axes[1, 1].bar(range(len(freehold_counts)), freehold_counts.values)
axes[1, 1].set_xticks(range(len(freehold_counts)))
axes[1, 1].set_xticklabels(freehold_counts.index)
axes[1, 1].set_title('Freehold vs Leasehold Properties', fontsize=12, fontweight='bold')
axes[1, 1].set_ylabel('Count')

plt.tight_layout()
plt.show()

### 2.6 PROCEDURE_EN Analysis (Critical for Filtering Decision)

**Why this matters:** The challenge asks us to predict market prices, but the dataset contains various transaction types (Sales, Mortgages, Gifts, etc.). We need to determine if non-sale transactions should be filtered out.

In [ ]:
# Analyze PROCEDURE_EN distribution
print("=" * 80)
print("PROCEDURE_EN (Transaction Type) ANALYSIS")
print("=" * 80)

procedure_counts = df['PROCEDURE_EN'].value_counts()
print("\nTransaction Type Distribution:")
print(procedure_counts)
print(f"\nTotal unique procedure types: {df['PROCEDURE_EN'].nunique()}")

In [ ]:
# Analyze TRANS_VALUE by PROCEDURE_EN
print("\n" + "=" * 80)
print("TRANS_VALUE STATISTICS BY PROCEDURE_EN")
print("=" * 80)

procedure_price_stats = df.groupby('PROCEDURE_EN')['TRANS_VALUE'].agg([
    'count', 'mean', 'median', 'std', 'min', 'max'
]).sort_values('count', ascending=False)

display(procedure_price_stats)

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Count by procedure
top_procedures = procedure_counts.head(10)
axes[0].barh(range(len(top_procedures)), top_procedures.values)
axes[0].set_yticks(range(len(top_procedures)))
axes[0].set_yticklabels(top_procedures.index)
axes[0].set_title('Top 10 Transaction Types by Count', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Count')

# Median price by procedure (top 10 by count)
top_procedure_names = top_procedures.index
median_prices = df[df['PROCEDURE_EN'].isin(top_procedure_names)].groupby('PROCEDURE_EN')['TRANS_VALUE'].median()
median_prices = median_prices.reindex(top_procedure_names)
axes[1].barh(range(len(median_prices)), median_prices.values, color='green')
axes[1].set_yticks(range(len(median_prices)))
axes[1].set_yticklabels(median_prices.index)
axes[1].set_title('Median Transaction Value by Type', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Median Transaction Value')

plt.tight_layout()
plt.show()

print("\n📊 DECISION POINT:")
print("   Based on the analysis above, we need to decide:")
print("   - Should we filter to only 'Sale' transactions?")
print("   - Or do multiple procedure types represent valid market prices?")
print("   - Look for procedure types with similar median prices to Sales")

### 2.7 Outlier Detection and Handling Strategy

**Approach:** Conservative, domain-driven outlier removal
- Remove only obvious data errors (impossible values)
- Keep legitimate luxury properties
- Use robust loss (MAE) to reduce sensitivity to remaining outliers

In [ ]:
# Identify potential outliers based on domain rules
print("=" * 80)
print("OUTLIER DETECTION (Domain-Driven Rules)")
print("=" * 80)

# Rule 1: Invalid areas (<=0)
invalid_actual_area = (df['ACTUAL_AREA'] <= 0).sum()
invalid_procedure_area = (df['PROCEDURE_AREA'] <= 0).sum()
print(f"\nRule 1 - Invalid Areas:")
print(f"  ACTUAL_AREA <= 0: {invalid_actual_area} rows ({invalid_actual_area/len(df)*100:.2f}%)")
print(f"  PROCEDURE_AREA <= 0: {invalid_procedure_area} rows ({invalid_procedure_area/len(df)*100:.2f}%)")

# Rule 2: Invalid transaction values (<=0)
invalid_trans_value = (df['TRANS_VALUE'] <= 0).sum()
print(f"\nRule 2 - Invalid Transaction Values:")
print(f"  TRANS_VALUE <= 0: {invalid_trans_value} rows ({invalid_trans_value/len(df)*100:.2f}%)")

# Rule 3: Extreme area values (will define thresholds based on percentiles)
area_p01 = df['ACTUAL_AREA'].quantile(0.001)
area_p99 = df['ACTUAL_AREA'].quantile(0.999)
print(f"\nRule 3 - Extreme Area Values:")
print(f"  0.1th percentile: {area_p01:.2f}")
print(f"  99.9th percentile: {area_p99:.2f}")
extreme_small_area = (df['ACTUAL_AREA'] < area_p01).sum()
extreme_large_area = (df['ACTUAL_AREA'] > area_p99).sum()
print(f"  Areas < {area_p01:.2f}: {extreme_small_area} rows")
print(f"  Areas > {area_p99:.2f}: {extreme_large_area} rows")

# Rule 4: Extreme price-per-area ratios
price_per_area_p01 = df['price_per_sqft_temp'].quantile(0.001)
price_per_area_p99 = df['price_per_sqft_temp'].quantile(0.999)
print(f"\nRule 4 - Extreme Price-per-Sqft Ratios:")
print(f"  0.1th percentile: {price_per_area_p01:.2f}")
print(f"  99.9th percentile: {price_per_area_p99:.2f}")
extreme_low_price_ratio = (df['price_per_sqft_temp'] < price_per_area_p01).sum()
extreme_high_price_ratio = (df['price_per_sqft_temp'] > price_per_area_p99).sum()
print(f"  Price/sqft < {price_per_area_p01:.2f}: {extreme_low_price_ratio} rows")
print(f"  Price/sqft > {price_per_area_p99:.2f}: {extreme_high_price_ratio} rows")

### 2.8 Key Business Insights

This section addresses the challenge requirements for insights on:
- Price trends by property type, area, and time
- Impact of location features
- Off-plan vs ready property pricing
- Freehold vs leasehold impact

In [ ]:
# 2.8.1 Price Trends by Property Type
print("=" * 80)
print("PRICE TRENDS BY PROPERTY TYPE")
print("=" * 80)

prop_type_stats = df.groupby('PROP_TYPE_EN').agg({
    'TRANS_VALUE': ['count', 'mean', 'median'],
    'ACTUAL_AREA': 'median'
}).round(2)

prop_type_stats.columns = ['Count', 'Mean_Price', 'Median_Price', 'Median_Area']
prop_type_stats = prop_type_stats.sort_values('Count', ascending=False).head(10)

display(prop_type_stats)

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

axes[0].barh(range(len(prop_type_stats)), prop_type_stats['Median_Price'])
axes[0].set_yticks(range(len(prop_type_stats)))
axes[0].set_yticklabels(prop_type_stats.index)
axes[0].set_title('Median Price by Property Type (Top 10)', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Median Transaction Value')

axes[1].barh(range(len(prop_type_stats)), prop_type_stats['Median_Area'], color='green')
axes[1].set_yticks(range(len(prop_type_stats)))
axes[1].set_yticklabels(prop_type_stats.index)
axes[1].set_title('Median Area by Property Type (Top 10)', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Median Area')

plt.tight_layout()
plt.show()

In [ ]:
# 2.8.2 Price Trends by Area
print("\n" + "=" * 80)
print("PRICE TRENDS BY AREA")
print("=" * 80)

area_stats = df.groupby('AREA_EN').agg({
    'TRANS_VALUE': ['count', 'mean', 'median'],
    'ACTUAL_AREA': 'median'
}).round(2)

area_stats.columns = ['Count', 'Mean_Price', 'Median_Price', 'Median_Area']
area_stats = area_stats[area_stats['Count'] >= 50].sort_values('Median_Price', ascending=False).head(15)

display(area_stats)

# Visualize top premium areas
plt.figure(figsize=(14, 8))
plt.barh(range(len(area_stats)), area_stats['Median_Price'])
plt.yticks(range(len(area_stats)), area_stats.index)
plt.title('Top 15 Premium Areas by Median Price (min 50 transactions)', fontsize=14, fontweight='bold')
plt.xlabel('Median Transaction Value')
plt.tight_layout()
plt.show()

print("\n📊 INSIGHT: Location premium is significant - top areas command much higher prices")

In [ ]:
# 2.8.3 Price Trends Over Time
print("\n" + "=" * 80)
print("PRICE TRENDS OVER TIME")
print("=" * 80)

# Monthly median prices
df['year_month'] = df['INSTANCE_DATE'].dt.to_period('M')
monthly_prices = df.groupby('year_month')['TRANS_VALUE'].agg(['median', 'count'])

# Quarterly median prices
df['year_quarter'] = df['INSTANCE_DATE'].dt.to_period('Q')
quarterly_prices = df.groupby('year_quarter')['TRANS_VALUE'].agg(['median', 'count'])

# Visualize
fig, axes = plt.subplots(2, 1, figsize=(16, 10))

# Monthly trend
axes[0].plot(monthly_prices.index.astype(str), monthly_prices['median'], marker='o', linewidth=2)
axes[0].set_title('Median Transaction Price - Monthly Trend', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Month')
axes[0].set_ylabel('Median Price')
axes[0].tick_params(axis='x', rotation=45)
axes[0].grid(True, alpha=0.3)

# Quarterly trend
axes[1].plot(quarterly_prices.index.astype(str), quarterly_prices['median'], marker='s', linewidth=2, color='green')
axes[1].set_title('Median Transaction Price - Quarterly Trend', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Quarter')
axes[1].set_ylabel('Median Price')
axes[1].tick_params(axis='x', rotation=45)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n📊 INSIGHT: Temporal patterns visible - justifies including time-based features")

In [ ]:
# 2.8.4 Off-Plan vs Ready Properties
print("\n" + "=" * 80)
print("OFF-PLAN VS READY PROPERTIES")
print("=" * 80)

offplan_comparison = df.groupby('IS_OFFPLAN_EN')['TRANS_VALUE'].agg([
    'count', 'mean', 'median', 'std'
]).round(2)

display(offplan_comparison)

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Box plot
df.boxplot(column='TRANS_VALUE', by='IS_OFFPLAN_EN', ax=axes[0])
axes[0].set_title('Transaction Value: Off-Plan vs Ready', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Property Status')
axes[0].set_ylabel('Transaction Value')
plt.sca(axes[0])
plt.xticks(rotation=0)

# Bar plot of medians
axes[1].bar(range(len(offplan_comparison)), offplan_comparison['median'])
axes[1].set_xticks(range(len(offplan_comparison)))
axes[1].set_xticklabels(offplan_comparison.index)
axes[1].set_title('Median Price: Off-Plan vs Ready', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Median Transaction Value')

plt.suptitle('')  # Remove default title
plt.tight_layout()
plt.show()

price_diff_pct = (offplan_comparison.loc['Off-Plan', 'median'] / offplan_comparison.loc['Ready', 'median'] - 1) * 100 if 'Ready' in offplan_comparison.index and 'Off-Plan' in offplan_comparison.index else 0
print(f"\n📊 INSIGHT: Off-plan properties are {abs(price_diff_pct):.1f}% {'more' if price_diff_pct > 0 else 'less'} expensive than ready properties (median)")

In [ ]:
# 2.8.5 Freehold vs Leasehold Impact
print("\n" + "=" * 80)
print("FREEHOLD VS LEASEHOLD PROPERTIES")
print("=" * 80)

freehold_comparison = df.groupby('IS_FREE_HOLD_EN')['TRANS_VALUE'].agg([
    'count', 'mean', 'median', 'std'
]).round(2)

display(freehold_comparison)

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Box plot
df.boxplot(column='TRANS_VALUE', by='IS_FREE_HOLD_EN', ax=axes[0])
axes[0].set_title('Transaction Value: Freehold vs Leasehold', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Ownership Type')
axes[0].set_ylabel('Transaction Value')
plt.sca(axes[0])
plt.xticks(rotation=0)

# Bar plot of medians
axes[1].bar(range(len(freehold_comparison)), freehold_comparison['median'], color='green')
axes[1].set_xticks(range(len(freehold_comparison)))
axes[1].set_xticklabels(freehold_comparison.index)
axes[1].set_title('Median Price: Freehold vs Leasehold', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Median Transaction Value')

plt.suptitle('')  # Remove default title
plt.tight_layout()
plt.show()

price_diff_pct_fh = (freehold_comparison.loc['Free Hold', 'median'] / freehold_comparison.loc['Lease Hold', 'median'] - 1) * 100 if 'Lease Hold' in freehold_comparison.index and 'Free Hold' in freehold_comparison.index else 0
print(f"\n📊 INSIGHT: Freehold properties are {abs(price_diff_pct_fh):.1f}% {'more' if price_diff_pct_fh > 0 else 'less'} expensive than leasehold properties (median)")

---
## 3. Data Cleaning and Preparation

Based on EDA insights, we'll now:
1. Filter dataset based on PROCEDURE_EN analysis
2. Remove obvious data errors (outliers)
3. Prepare data for feature engineering

In [ ]:
# Store original dataset size
original_size = len(df)
print(f"Original dataset size: {original_size:,} rows")

# TODO: Based on PROCEDURE_EN analysis above, decide filtering strategy
# For now, we'll create a placeholder that will be filled after EDA analysis
# Example: df_clean = df[df['PROCEDURE_EN'] == 'Sales'].copy()

print("\n⚠️  DECISION REQUIRED: Review PROCEDURE_EN analysis above to determine filtering strategy")

---
## Status: EDA Framework Complete ✓

**Next Steps:**
1. Run this notebook and analyze results
2. Make decisions on PROCEDURE_EN filtering
3. Define exact outlier removal rules
4. Proceed to feature engineering section

**The notebook will continue with:**
- Feature Engineering
- Model Training
- Evaluation & Error Analysis
- SHAP Interpretability

This will be added in the next iteration after we analyze the EDA results.